In [7]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyarrow as pa
import pyarrow.parquet as pq

In [102]:
fund_codes = ['MMM','ABT','ABBV']
start_date = '2018-11-30'
end_date = '2020-04-30'

In [103]:
def loadHistoricalIndex(fund_codes, start_date, end_date):
    response_columns = fund_codes
    response_columns = ["date"] + fund_codes
    all_historical_prices = pq.read_table('dailyPrice.parquet', columns= response_columns).to_pandas()
    subset_data = all_historical_prices[
        all_historical_prices["date"] >= start_date
    ][all_historical_prices["date"] <= end_date]
    subset_data = subset_data.sort_values("date").reset_index(drop=True)
    for i in fund_codes:
        subset_data[f"{i}index"] = subset_data[i] / subset_data[i][0]

    subset_data = subset_data.fillna(0)
    subset_data = subset_data.drop(fund_codes, axis=1)
    subset_data.columns = response_columns
    
    return json.loads(subset_data.to_json(orient="records"))


In [104]:
historical_index = loadHistoricalIndex(fund_codes, start_date, end_date)

In [105]:
portfolio= [ {"fund": "MMM", "amount": 1000}, 
{"fund": "ABT", "amount": 1000},
{"fund":"ABBV", "amount":1000}]
strategy= {'rebalance': True, "rebalanceFrequency": "m"}

In [106]:
rebalanceFrequency = strategy['rebalanceFrequency'].lower()

In [107]:
num_funds = len(portfolio)

In [108]:
fund_index = pd.DataFrame(historical_index)
fund_list = []
fund_amount = []
for i in portfolio:
    fund_list.append(i["fund"])
    fund_amount.append(i["amount"])
fund_percentage = [i /sum(fund_amount) for i in fund_amount]

In [109]:
def backmodel (fund_amount, fund_codes, fund_index):
    fund_diag = np.zeros((num_funds,num_funds))
    np.fill_diagonal(fund_diag, fund_amount)
    return fund_index[fund_codes].dot(fund_diag)

In [110]:
result_df = backmodel (fund_amount, fund_codes, fund_index)
result_df.columns = fund_codes
result_df['portfolio'] = result_df.sum(axis = 1)

In [140]:
fund_codes = ['MMM','ABT','ABBV']
start_date = '2018-11-30'
end_date = '2020-04-30'

In [141]:
frequency = {"y": 12, "q": 3, "m": 1}
start_period = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')
delta_months = relativedelta(end_date,start_period)

In [142]:
delta_months

relativedelta(years=+1, months=+5)

In [143]:
num_rebalance = int (diff_months / frequency[rebalanceFrequency])
if diff_months / frequency[rebalanceFrequency] - num_rebalance :
    num_rebalance = num_rebalance + 1

In [145]:
diff_months = delta_months.months + delta_months.years*12
if delta_months.days != 0:
    diff_months = diff_months + 1

In [146]:
temp_amount = fund_amount
subset_list= []

In [147]:
start_period + relativedelta(months=frequency[rebalanceFrequency])

datetime.datetime(2018, 12, 30, 0, 0)

In [148]:
diff_months /frequency[rebalanceFrequency]

17.0

In [149]:
for j in range(0, num_rebalance):
    end_period = start_period + relativedelta(months=frequency[rebalanceFrequency])
    start_period = start_period.strftime('%Y-%m-%d')
    end_period = end_period.strftime('%Y-%m-%d')

    subset_index = fund_index[fund_index['date'] >= start_period][fund_index['date'] <= end_period]
    for k in fund_codes:
        subset_index[k] = subset_index[k] / subset_index[k][subset_index.index[0]]
    temp_dates = subset_index['date']

    temp_df = backmodel(temp_amount, fund_codes, subset_index)
    temp_total = temp_df.iloc[(temp_df.shape[0] - 1)].sum()
    temp_amount = [temp_total * i for i in fund_percentage]
    temp_df["date"] = temp_dates

    start_period = datetime.strptime(
        temp_df["date"][temp_df.index[-1]], "%Y-%m-%d"
    )

    temp_df.drop(temp_df.tail(1).index, inplace=True)
    subset_list.append(temp_df)

In [150]:
subset_list

[              0            1            2        date
 0   1000.000000  1000.000000  1000.000000  2018-11-30
 1   1004.040015  1002.970966   966.794790  2018-12-03
 2    972.489419   965.563808   964.445868  2018-12-04
 3    974.797999   966.103984   928.464659  2018-12-06
 4    953.443632   944.632005   935.404655  2018-12-07
 5    953.828395   957.731263   941.383728  2018-12-10
 6    949.692189   970.155300   945.974803  2018-12-11
 7    960.994613   979.473329   936.472347  2018-12-12
 8    972.152751   974.746793   914.050822  2018-12-13
 9    943.151212   955.570560   906.790519  2018-12-14
 10   927.375914   941.931128   892.376682  2018-12-17
 11   935.744517   946.387576   891.736067  2018-12-18
 12   913.620623   931.802836   911.488362  2018-12-19
 13   901.212005   923.430115   906.683750  2018-12-20
 14   883.753367   908.440243   898.569293  2018-12-21
 15   859.080416   885.347738   950.672646  2018-12-24
 16   895.825317   940.175557   959.961563  2018-12-26
 17   917.

In [ ]:
fund_index["portfolio"] = fund_index[fund_list].sum(axis=1)
json.loads(fund_index[["date", "portfolio"]].to_json(orient="records"))